In [ ]:
import numpy as np

def getdJdOmega(Z_K, ypsilon, c, eta_div, omega, mu):
    dJdOmega = Z_K @ ((ypsilon - c) * eta_div( np.transpose(Z_K) @ omega + mu)) #ha med np.transpose??
    return dJdOmega

def getdJdW(h,P,sigma_div, W, b, Z, K): #Ha med K??
    dJdW = np.zeros_like(W)
    #K = len(W)
    for k in range(K):
        dJdW[k] = h*(P[k+1]*sigma_div(W[k]@Z[k]+b[k]))@np.transpose(Z[k])
    return dJdW


def getdJdb(h,P,sigma_div, W, b, Z, K): #Ha med K??
    dJdb = np.zeros_like(b)
    dJdb = np.zeros_like(b)
    #K = len(W)
    for k in range(K):
        dJdb[k] = h*(P[k+1]*sigma_div(W[k]@Z[k]+b[k]))@np.transpose(np.array([[1]*len(Z[0,0,:])]))
    return dJdb


def getYpsilon(eta, Z_K, omega, mu):
    return eta(np.transpose(Z_K) @ omega + mu)

In [ ]:
def getZ(Y,K,sigma,W,b,h):
    #shape parameters
    K = W.shape[0]
    d = Y.shape[0]
    I = Y.shape[1]

    #initialize Z, Z0=Y
    Z = np.zeros((K,d,I))
    Z[0] = Y

    #finds Zk
    for k in range(1,K):
        u = W[k-1]@Z[k-1] + b[k-1]
        Z[k] = Z[k-1] + h*sigma(u)

    #evt legge inn her å finn ypsilon? idk

    return Z

def getP(omega,ypsilon,c,etadiff,mu,h,W,sigmadiff,b,Z):
    #shape parameters
    K = W.shape[0]
    d = Y.shape[0]
    I = Y.shape[1]

    #initialize P
    P = np.zeros((K+1,d,I))

    #finds P_K
    u = np.transpose(Z[-1])@omega + np.full((d,I),mu)
    P[-1]= omega@np.transpose((ypsilon-c)*etadiff(u))

    #finds P_K-1 to P_0
    for k in range(K,-1,-1):
        s = W[k]@Z[k] + b[k]
        P[k]=P[k+1] + h*np.transpose(W[k])@(sigmadiff(s)*P[k+1])

    return P

def getdJdmu(Zk,ypsilon,omega,mu,c,etadiff):
    u = np.transpose(Zk)@omega + np.full(omega.shape,mu)

    return np.transpose(etadiff(u))*(ypsilon-c)